<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Exploratory Data Analysis with SQL
Estimated time needed: 60 minutes


# Introduction
Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection
Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.  
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [2]:
# provide your solution here
install.packages("RODBC")
library(RODBC)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [3]:
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"          
dsn_hostname <- "8e359033-a1c9-4643-82ef-8ac06f5107eb.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" 
dsn_port <- "30120"  
dsn_protocol <- "TCPIP"       
dsn_uid <- "dvt13187"       
dsn_pwd <- "XKfEmM8pc5Ay1bTh"     
dsn_security <- "ssl"

conn_path <- paste("DRIVER=",dsn_driver,
                   ";DATABASE=",dsn_database,
                   ";HOSTNAME=",dsn_hostname,
                   ";PORT=",dsn_port,
                   ";PROTOCOL=",dsn_protocol,
                   ";UID=",dsn_uid,
                   ";PWD=",dsn_pwd,
                   ";SECURITY=",dsn_security,        
                   sep="")

conn <- odbcDriverConnect(conn_path)
conn 

RODBC Connection 1
Details:
  case=nochange
  DRIVER={IBM DB2 ODBC DRIVER}
  UID=dvt13187
  PWD=******
  DATABASE=bludb
  HOSTNAME=8e359033-a1c9-4643-82ef-8ac06f5107eb.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
  PORT=30120
  PROTOCOL=TCPIP
  SECURITY=SSL

In [4]:
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

DBMS_Name 
"DB2/LINUXX8664"

DBMS_Ver 
"11.05.0800"

Driver_ODBC_Ver 
        "03.51"

In [5]:
# check my table
tab.frame <- sqlTables(conn,schema='DVT13187')
nrow(tab.frame)
tab.frame$TABLE_NAME

[1] 4

[1] "BIKE_SHARING_SYSTEMS"    "CITIES_WEATHER_FORECAST"
[3] "SEOUL_BIKE_SHARING"      "WORLD_CITIES"


## Task 1 - Record Count
#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [6]:
# provide your solution here
query = "SELECT COUNT (*)
FROM SEOUL_BIKE_SHARING"
sqlQuery(conn,query)

,1
,<int>
1,8465


## Task 2 - Operational Hours
#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [7]:
# provide your solution here
query = "SELECT SUM(HOUR) 
FROM seoul_bike_sharing
WHERE rented_bike_count > 0"
sqlQuery(conn,query)

,1
,<int>
1,97407


## Task 3 - Weather Outlook
#### Query the the weather forecast for Seoul over the next 3 hours.
Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [8]:
# provide your solution here
query = "select *
from cities_weather_forecast 
where CITY = 'Seoul'
limit  1"
sqlQuery(conn,query)

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


## Task 4 - Seasons
#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [9]:
# provide your solution here
query = "
SELECT DISTINCT SEASONS 
FROM seoul_bike_sharing;
"
sqlQuery(conn,query)

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range
#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [10]:
# provide your solution here
query = "
SELECT MIN(DATE) as First_Date, MAX(DATE) as Last_Date
FROM SEOUL_BIKE_SHARING;
"
sqlQuery(conn,query)

,FIRST_DATE,LAST_DATE
,<fct>,<fct>
1,01/01/2018,31/12/2017


## Task 6 - Subquery - 'all-time high'
#### determine which date and hour had the most bike rentals.


### Solution 6


In [11]:
# provide your solution here
query = "
SELECT DATE, HOUR, RENTED_BIKE_COUNT
FROM seoul_bike_sharing
WHERE RENTED_BIKE_COUNT = (SELECT MAX(RENTED_BIKE_COUNT) FROM seoul_bike_sharing);
"
sqlQuery(conn,query)

,DATE,HOUR,RENTED_BIKE_COUNT
,<fct>,<int>,<int>
1,19/06/2018,18,3556


## Task 7 - Hourly popularity and temperature by season
#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [12]:
# provide your solution here
query = "
SELECT SEASONS, HOUR, AVG(TEMPERATURE) as avg_temp, AVG(RENTED_BIKE_COUNT) as avg_rent
FROM seoul_bike_sharing
GROUP BY SEASONS, HOUR
ORDER BY avg_rent DESC
LIMIT 10;
"
sqlQuery(conn,query)

,SEASONS,HOUR,AVG_TEMP,AVG_RENT
,<fct>,<int>,<dbl>,<int>
1,Summer,18,29.38696,2135
2,Autumn,18,16.03086,1983
3,Summer,19,28.27283,1889
4,Summer,20,27.06630,1801
5,Summer,21,26.27826,1754
6,Spring,18,15.97222,1689
7,Summer,22,25.69891,1567
8,Autumn,17,17.27778,1562
9,Summer,17,30.07500,1526


## Task 8 - Rental Seasonality
#### Find the average hourly bike count during each season. 
Also include the minimum, maximum, and standard deviation of the hourly bike count for each season. 


### Solution 8


In [13]:
# provide your solution here
query = "
SELECT SEASONS, 
       AVG (RENTED_BIKE_COUNT) as avg_bike_count,
       MIN(RENTED_BIKE_COUNT) as min_bike_count,
       MAX (RENTED_BIKE_COUNT) as max_bike_count,
       STDDEV (RENTED_BIKE_COUNT) as std_dev_bike_count
FROM seoul_bike_sharing
GROUP BY  SEASONS "
sqlQuery(conn,query)

,SEASONS,AVG_BIKE_COUNT,MIN_BIKE_COUNT,MAX_BIKE_COUNT,STD_DEV_BIKE_COUNT
,<fct>,<int>,<int>,<int>,<dbl>
1,Autumn,924,2,3298,617.3885
2,Spring,746,2,3251,618.5247
3,Summer,1034,9,3556,690.0884
4,Winter,225,3,937,150.3374


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality
#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?
Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all. 


### Solution 9


In [14]:
# provide your solution here
query = "
SELECT SEASONS, AVG(RENTED_BIKE_COUNT) as avg_bike_count, AVG(TEMPERATURE) as avg_temp, 
   AVG(HUMIDITY) as avg_humidity, AVG(WIND_SPEED) as avg_wind_speed,
   AVG(VISIBILITY) as avg_visibility, AVG(DEW_POINT_TEMPERATURE) as avg_dew_point_temp,
   AVG(SOLAR_RADIATION) as avg_solar_radiation, AVG(RAINFALL) as avg_rainfall,
   AVG(SNOWFALL) as avg_snowfall
 FROM seoul_bike_sharing
 GROUP BY SEASONS
 ORDER BY avg_bike_count DESC
           "
sqlQuery(conn,query)

,SEASONS,AVG_BIKE_COUNT,AVG_TEMP,AVG_HUMIDITY,AVG_WIND_SPEED,AVG_VISIBILITY,AVG_DEW_POINT_TEMP,AVG_SOLAR_RADIATION,AVG_RAINFALL,AVG_SNOWFALL
,<fct>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Summer,1034,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000
2,Autumn,924,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026
3,Spring,746,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000
4,Winter,225,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000


## Task 10 - Total Bike Count and City Info for Seoul
#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.
Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [15]:
# provide your solution here

query = "
SELECT WORLD_CITIES.CITY, WORLD_CITIES.COUNTRY, WORLD_CITIES.LAT,
   WORLD_CITIES.LNG, WORLD_CITIES.POPULATION, 
   SUM(BIKE_SHARING_SYSTEMS.BICYCLES) AS TOTAL_BIKES
FROM WORLD_CITIES 
 JOIN BIKE_SHARING_SYSTEMS 
  ON WORLD_CITIES.CITY_ASCII = BIKE_SHARING_SYSTEMS.CITY
WHERE WORLD_CITIES.CITY = 'Seoul' 
GROUP BY WORLD_CITIES.CITY, WORLD_CITIES.COUNTRY,
WORLD_CITIES.LAT, WORLD_CITIES.LNG, WORLD_CITIES.POPULATION;
           "
sqlQuery(conn,query)

,CITY,COUNTRY,LAT,LNG,POPULATION,TOTAL_BIKES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Seoul,"Korea, South",37.58,127,21794000,20000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system
#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.
Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [20]:
# provide your solution here
# I CHANGE THE "NA" TO 0 CAUSE I TRY EVERYTHING TO FILTER BUT DONT WORK IN IBM DB2
## i use this code to do that
### ( UPDATE BIKE_SHARING_SYSTEMS
###  SET BICYCLES = 0
###  WHERE BICYCLES = 'NA';)

query = "
SELECT wc.CITY AS CITY, wc.COUNTRY AS COUNTRY, wc.LAT AS LAT, wc.LNG AS LNG, wc.POPULATION AS POPULATION, SUM(bs.BICYCLES) AS BICYCLES
FROM WORLD_CITIES wc
JOIN BIKE_SHARING_SYSTEMS bs ON wc.CITY = bs.CITY
WHERE bs.BICYCLES NOT LIKE 'NA' AND bs.BICYCLES IS NOT NULL
GROUP BY wc.CITY, wc.COUNTRY, wc.LAT, wc.LNG, wc.POPULATION
HAVING SUM(bs.BICYCLES) BETWEEN 15000 AND 20000
       "
sqlQuery(conn,query)

,CITY,COUNTRY,LAT,LNG,POPULATION,BICYCLES
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>
1,Beijing,China,39.90,116.39,19433000,16000
2,Milan,United States,35.91,-88.75,7201,17000
3,Milan,United States,42.08,-83.68,7778,17000
4,Milan,Italy,45.46,9.19,1351562,17000
5,Ningbo,China,29.87,121.54,7639000,15000
6,Seoul,"Korea, South",37.58,127.00,21794000,20000
7,Shanghai,China,31.16,121.46,22120000,19165
8,Weifang,China,36.71,119.10,9373000,20000
9,Zhuzhou,China,27.84,113.14,3855609,20000


In [21]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                |
| ---------- | ------- | ------------- | --------------------------------------------------------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                               |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review                               |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins                                  |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation                                                          |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
